In [75]:
from google import genai
import os
from dotenv import load_dotenv
import duckdb
load_dotenv()
from google import generativeai as oldGenai
from google.generativeai.types import GenerationConfig

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
with duckdb.connect("../../ads_data.duckdb") as con: 
    df_mart = con.execute("SELECT * FROM mart.mart_kultur_media").df()
top_employer = df_mart.groupby("occupation")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()

with open("for_gemini.txt", "r", encoding='utf-8') as file: 
    ad_text = file.read() 

max_rows = 100
ad_text_splitted = ad_text.split("\n") #tagit bort alla mellanslag
chunks = [] # lista för textbitar

for i in range(0, len(ad_text_splitted), max_rows): 
    chunk = "\n".join(ad_text_splitted[i:i+max_rows])
    chunks.append(chunk)


response_list = []
for chunk in chunks:
    promt = f"""Du är en rekryterare inom Media, Kultur, Design.
    Plocka ut max 3 av vanligaste kraven och max av 3 vanligaste meriterade och antal av dessa som efterfrågas i dessa jobbannonser, samt en sammanfattning av krav och meriter som du skriver ut (om det finns):
    {chunk}

    Output ska vara i detta formatet enbart: 

    {{
        "sammanfattning": ["sammanfattning"]
        "krav": [erfarenhet1, erfarenhet2, ...]
        "antal krav": [summa erfarenhet1..]
        "meriterande": [meriterande1,meriterande2, ... ]
        "antal meriterande": [summa meriterande1..]
        
    }}
    """
    model = oldGenai.GenerativeModel(
        model_name="gemini-2.0-flash",
        generation_config=GenerationConfig(
            temperature=0.0,         
            top_p=1.0,  # Säkerställer att sampling inte påverkas   #verkar inte göra skillnad
            top_k=1     # Tar bort variation)                       #verkar inte göra skillnad    
        )
    )
    response = model.generate_content(promt)
    print(response.text)
    response_list.append(response.text)

```json
{
    "sammanfattning": [
        "Samtliga annonser söker engagerade och drivna journalister/reportrar med god stilistisk förmåga och förmåga att knyta kontakter. Digital kompetens och intresse för lokaljournalistik är genomgående viktiga."
    ],
    "krav": [
        "Journalistutbildning eller motsvarande arbetslivserfarenhet",
        "Tidigare arbete på nyhetsredaktion",
        "B-körkort"
    ],
    "antal krav": [
        "3",
        "2",
        "1"
    ],
    "meriterande": [
        "Erfarenhet av granskande journalistik",
        "Erfarenhet av bildjournalistik",
        "Lokal kännedom"
    ],
    "antal meriterande": [
        "2",
        "2",
        "1"
    ]
}
```
```json
{
    "sammanfattning": [
        "Samtliga annonser söker engagerade och nyfikna personer som vill arbeta med journalistik och nyhetsförmedling. Fokus ligger på att hitta och berätta nyheter, ofta med en lokal anknytning och med fokus på att beröra och engagera publiken. Digital kompetens 

In [ ]:
import pandas as pd
import json
data = response.text
cleaned = data.strip("```json") #.replace("\n", "").strip().replace("  ", "")
data = json.loads(cleaned)
for chunk in data: 
    response_dict = pd.DataFrame({
        "sammanfattning": data.get("sammanfattning")
    })
response_dict["sammanfattning"] # hämtar ut den sista sammanfattningen

0    Sveriges Radio söker personer med intresse för...
Name: sammanfattning, dtype: object

In [89]:
# model2 = oldGenai.GenerativeModel(
#     model_name="gemini-2.0-flash",
#     generation_config=GenerationConfig(
#         temperature=0.0,         
#         top_p=1.0,  # Säkerställer att sampling inte påverkas   #verkar inte göra skillnad
#         top_k=1     # Tar bort variation)                       #verkar inte göra skillnad    
#     )
# )

promt2 = f"""kan du analysera detta och summera till en enda dictionary: 
{response_list}

Output ska vara i detta formatet enbart: 

    {{
        "sammanfattning": ["sammanfattning"]
        "krav": [erfarenhet1, erfarenhet2, ...]
        "antal krav": [summa erfarenhet1..]
        "meriterande": [meriterande1,meriterande2, ... ]
        "antal meriterande": [summa meriterande1..]
        
    }}
    """
response2 = model.generate_content(promt2)
print(response2.text)

```python
import json

data = ['```json\n{\n    "sammanfattning": [\n        "Samtliga annonser söker engagerade och drivna journalister/reportrar med god stilistisk förmåga och förmåga att knyta kontakter. Digital kompetens och intresse för lokaljournalistik är genomgående viktiga."\n    ],\n    "krav": [\n        "Journalistutbildning eller motsvarande arbetslivserfarenhet",\n        "Tidigare arbete på nyhetsredaktion",\n        "B-körkort"\n    ],\n    "antal krav": [\n        "3",\n        "2",\n        "1"\n    ],\n    "meriterande": [\n        "Erfarenhet av granskande journalistik",\n        "Erfarenhet av bildjournalistik",\n        "Lokal kännedom"\n    ],\n    "antal meriterande": [\n        "2",\n        "2",\n        "1"\n    ]\n}\n```', '```json\n{\n    "sammanfattning": [\n        "Samtliga annonser söker engagerade och nyfikna personer som vill arbeta med journalistik och nyhetsförmedling. Fokus ligger på att hitta och berätta nyheter, ofta med en lokal anknytning och m

In [ ]:
import pandas as pd
import json
data = response2.text
cleaned = data.strip("```json") #.replace("\n", "").strip().replace("  ", "")
data = json.loads(cleaned)
sammanfattning = data["sammanfattning"]
sammanfattning

In [ ]:
df_krav = pd.DataFrame({
    "krav": data["krav"], 
    "antal krav": data["antal krav"]
})
df_krav

,krav,antal krav
0,Journalistutbildning eller motsvarande arbetsl...,6
1,Erfarenhet av journalistiskt arbete på nyhetsr...,2
2,B-körkort,3
3,Förmåga att analysera trafiksiffror och läsarb...,1
4,Flera års erfarenhet som redaktör/nyhetschef p...,1
5,Relevant utbildning och/eller erfarenhet från ...,1
6,Erfarenhet av redaktionellt nyhetsarbete,11
7,Gedigen digital kompetens,1
8,"Stort intresse av samhällsfrågor, kultur och n...",9
9,Minst gymnasieutbildning,5


In [ ]:
df_merit = pd.DataFrame({
    "meriterande": data["meriterande"],
    "antal meriterande": data["antal meriterande"]
})
df_merit

,meriterande,antal meriterande
0,Erfarenhet av granskande journalistik,1
1,Erfarenhet av bildjournalistik,1
2,Lokal kännedom,3
3,Erfarenhet av personalansvar,1
4,Gott publicistiskt omdöme och vana att diskute...,1
5,Fackkunskaper,1
6,Erfarenhet av undersökande journalistik,1
7,God förmåga att uttrycka sig i tal och skrift,1
8,Brett kontaktnät,1
9,Eftergymnasial utbildning,7


In [87]:
with open("embedding.txt", "r") as file:
    text = file.read()
promt3 = f"Kan du säga vad det står här: {text}"
response3 = model.generate_content(promt3)
response3.text

'Det är en lista med 300 flyttal (decimaltal) i vetenskaplig notation (även kallad exponentiell notation).  "e-02" betyder "gånger 10 upphöjt till -2", vilket är samma sak som att dividera med 100.\n\nHär är några exempel på hur man tolkar dem:\n\n*   `-1.374748535454273224e-02`  är  `-0.01374748535454273224`\n*   `2.319343388080596924e-02`  är  `0.02319343388080596924`\n*   `1.300993561744689941e-01` är `0.1300993561744689941`\n\nMed andra ord, flyttalen är representerade som:\n\n`mantissa * 10^(exponent)`\n\nDär mantissan är talet före "e" och exponenten är talet efter "e".\n\nListan verkar vara en serie numeriska värden, men utan ytterligare kontext är det svårt att säga vad de representerar. De kan vara data från en sensor, resultat från en beräkning, eller något annat.\n'